### Libraries

In [1]:
import numpy as np
import pandas as pd

import random
import math
import datetime

import copy
import json
from heapq import nsmallest
from collections import OrderedDict 

# Shortest Path Algorithm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import floyd_warshall

### Time Functions

In [3]:
def time2timedelta(timeObj):
    return datetime.datetime.combine(datetime.date.min, timeObj) - datetime.datetime.min

def timedelta2time(deltaObj):
    return (datetime.datetime.min + deltaObj).time()

def datetime2time(timeObj):
    return datetime.datetime.strptime(timeObj.strftime('%H:%M:%S'),'%H:%M:%S').time()

### Supply Parameters

In [4]:
Bus_Capacity = 10
Fleet_Size = 10
Uniform_Speed = 50
Waiting_Time_Threshold = time2timedelta(datetime.time(0,30)) # 30 minutes
Travel_Time_Threshold = time2timedelta(datetime.time(0,30)) # 30 minutes

In [5]:
def toTime(random_number):
    hours = math.floor(random_number/60)
    minutes = math.floor(random_number) if random_number < 60 else (math.floor(random_number) - (hours*60))
    seconds = round(((random_number - math.floor(random_number))*0.6)*100)
    x = timedelta2time(datetime.timedelta(hours = hours, minutes = minutes, seconds = seconds))
    return x 

def timeCalculation(Uniform_Speed, Distance):
    return (Distance/Uniform_Speed)*60

print(timeCalculation(Uniform_Speed, 5))
toTime(timeCalculation(Uniform_Speed, 5))

6.0


datetime.time(0, 6)

### Grid Of The City

In [6]:
loc = "ABCDEFGHIJKLMNOPQRSTUVWXY"
temp, locations = [], []
index = (np.arange(25)+1).reshape(5,5).tolist()

for ind, letter in enumerate(loc):
    temp.append(letter)
    if (ind+1)%5 == 0:
        locations.append(temp)
        temp = []
locations

[['A', 'B', 'C', 'D', 'E'],
 ['F', 'G', 'H', 'I', 'J'],
 ['K', 'L', 'M', 'N', 'O'],
 ['P', 'Q', 'R', 'S', 'T'],
 ['U', 'V', 'W', 'X', 'Y']]

### Distances From Each City

In [7]:
Loc = list(loc)
numpy_data = np.zeros((len(Loc),len(Loc))).tolist()

distance_grid = pd.DataFrame(data=numpy_data, index=Loc, columns=Loc)

for i, row in enumerate(locations):
    for j, area in enumerate(row):
        
        place = locations[i][j]

        if i-1 < 0 and j-1 < 0:
            temp = [locations[i+1][j], locations[i][j+1], locations[i+1][j+1]]
        elif i-1 < 0 and j+1 >= len(row):
            temp = [locations[i+1][j], locations[i][j-1], locations[i+1][j-1]]
        elif i+1 >= len(locations) and j-1 < 0:
            temp = [locations[i-1][j], locations[i][j+1], locations[i-1][j+1]]
        elif i+1 >= len(locations) and j+1 >= len(row):
            temp = [locations[i-1][j], locations[i][j-1], locations[i-1][j-1]]
        elif i-1 < 0:
            temp = [locations[i][j-1], locations[i][j+1], locations[i+1][j-1], locations[i+1][j+1], locations[i+1][j]]
        elif j-1 < 0:
            temp = [locations[i-1][j], locations[i+1][j], locations[i-1][j+1], locations[i+1][j+1], locations[i][j+1]]
        elif j+1 >= len(row):
            temp = [locations[i-1][j], locations[i+1][j], locations[i-1][j-1], locations[i+1][j-1], locations[i][j-1]]
        elif i+1 >= len(locations):
            temp = [locations[i][j-1], locations[i][j+1], locations[i-1][j-1], locations[i-1][j+1], locations[i-1][j]]
        else:
            temp = [locations[i][j-1], locations[i][j+1], locations[i-1][j-1], locations[i-1][j+1], locations[i-1][j], locations[i+1][j-1], locations[i+1][j+1], locations[i+1][j]]
        
        
        for change in temp:
                distance_grid[place][change] = 5.0
distance_grid

,A,B,C,D,E,F,G,H,I,J,...,P,Q,R,S,T,U,V,W,X,Y
A,0.0,5.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,5.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C,0.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
D,0.0,0.0,5.0,0.0,5.0,0.0,0.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
F,5.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
G,5.0,5.0,5.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
H,0.0,5.0,5.0,5.0,0.0,0.0,5.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I,0.0,0.0,5.0,5.0,5.0,0.0,0.0,5.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
J,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Importing Travel Demand

In [8]:
TD = pd.read_csv('Simulation_Input_750.csv')
TD['Entry Time'] = pd.to_datetime(TD['Entry Time'])
TD.dtypes
N = len(TD)
TD

,Passengers,Entry Time,Passenger Origin,Passenger Destination
0,1,2009-01-01 09:00:25,E,Y
1,2,2009-01-01 09:00:29,K,P
2,3,2009-01-01 09:00:50,E,G
3,4,2009-01-01 09:02:40,E,G
4,5,2009-01-01 09:02:48,X,A
...,...,...,...,...
745,746,2009-01-01 21:25:38,J,S
746,747,2009-01-01 21:25:46,E,T
747,748,2009-01-01 21:27:30,T,R
748,749,2009-01-01 21:28:12,G,P


### Shortest Path Algorithm

In [9]:
def ConstructPath(p, i, j):
    path = []
    while i!=j:
        path.append(Index2Loc[j])
        j = p[i,j]
    path.append(Index2Loc[j])
    return path[::-1]

In [10]:
graph = distance_grid.values.tolist()
graph = csr_matrix(graph)
dist_matrix, predecessors = floyd_warshall(csgraph=graph, directed=False, return_predecessors=True)
# dist_matrix # NxN matrix with the smallest distances between 2 locations
# predecessors # To help reconstruct the path for shortest algorithm
dist_matrix

array([[ 0.,  5., 10., 15., 20.,  5.,  5., 10., 15., 20., 10., 10., 10.,
        15., 20., 15., 15., 15., 15., 20., 20., 20., 20., 20., 20.],
       [ 5.,  0.,  5., 10., 15.,  5.,  5.,  5., 10., 15., 10., 10., 10.,
        10., 15., 15., 15., 15., 15., 15., 20., 20., 20., 20., 20.],
       [10.,  5.,  0.,  5., 10., 10.,  5.,  5.,  5., 10., 10., 10., 10.,
        10., 10., 15., 15., 15., 15., 15., 20., 20., 20., 20., 20.],
       [15., 10.,  5.,  0.,  5., 15., 10.,  5.,  5.,  5., 15., 10., 10.,
        10., 10., 15., 15., 15., 15., 15., 20., 20., 20., 20., 20.],
       [20., 15., 10.,  5.,  0., 20., 15., 10.,  5.,  5., 20., 15., 10.,
        10., 10., 20., 15., 15., 15., 15., 20., 20., 20., 20., 20.],
       [ 5.,  5., 10., 15., 20.,  0.,  5., 10., 15., 20.,  5.,  5., 10.,
        15., 20., 10., 10., 10., 15., 20., 15., 15., 15., 15., 20.],
       [ 5.,  5.,  5., 10., 15.,  5.,  0.,  5., 10., 15.,  5.,  5.,  5.,
        10., 15., 10., 10., 10., 10., 15., 15., 15., 15., 15., 15.],
      

In [11]:
Index2Loc = {np.arange(25).tolist()[i]: Loc[i] for i in range(len(Loc))}
Loc2Index = {value:key for key, value in Index2Loc.items()} 
Loc2Index

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24}

### Calculating Shortest Path

In [12]:
columnA, columnB = [], []
for i in loc:
    for j in loc:
        if j == i:
            continue
        columnA.append(i)
        columnB.append(j)

SP = pd.DataFrame()
SP['Origin'] = columnA
SP['Exit'] = columnB
SP.head(5)

,Origin,Exit
0,A,B
1,A,C
2,A,D
3,A,E
4,A,F


In [13]:
shortestPath = []
for i in range(len(SP)):
    spArr = ConstructPath(predecessors, Loc2Index[SP.iloc[i]['Origin']], Loc2Index[SP.iloc[i]['Exit']])
    shortestPath.append(spArr)

SP['Shortest Path'] = shortestPath
SP.head()

,Origin,Exit,Shortest Path
0,A,B,"[A, B]"
1,A,C,"[A, B, C]"
2,A,D,"[A, B, C, D]"
3,A,E,"[A, B, C, D, E]"
4,A,F,"[A, F]"


In [14]:
shortestPath = []
distance =[]

for i in range(len(TD)):
    O = TD.iloc[i]['Passenger Origin']
    D = TD.iloc[i]['Passenger Destination']
    arr = SP.iloc[SP[(SP['Origin'] == O) & (SP['Exit'] == D)].index[0]]['Shortest Path']
    distance.append(dist_matrix[Loc2Index[O]][Loc2Index[D]])
    shortestPath.append(arr)
    
TD['Shortest Path'] = shortestPath
TD['Distance'] = distance
TD

,Passengers,Entry Time,Passenger Origin,Passenger Destination,Shortest Path,Distance
0,1,2009-01-01 09:00:25,E,Y,"[E, I, M, S, Y]",20.0
1,2,2009-01-01 09:00:29,K,P,"[K, P]",5.0
2,3,2009-01-01 09:00:50,E,G,"[E, D, C, G]",15.0
3,4,2009-01-01 09:02:40,E,G,"[E, D, C, G]",15.0
4,5,2009-01-01 09:02:48,X,A,"[X, R, L, F, A]",20.0
...,...,...,...,...,...,...
745,746,2009-01-01 21:25:38,J,S,"[J, N, S]",10.0
746,747,2009-01-01 21:25:46,E,T,"[E, I, N, T]",15.0
747,748,2009-01-01 21:27:30,T,R,"[T, N, R]",10.0
748,749,2009-01-01 21:28:12,G,P,"[G, K, P]",10.0


### Helping Functions

In [15]:
def second_smallest(numbers):
    return nsmallest(2, numbers)[-1]
second_smallest([1,2,3,4,5,6])

2

In [16]:
def sortKeys(d):
    d = OrderedDict(sorted(d.items())) 
    return {**d}

def sortValues(d):
    d = OrderedDict(sorted(d.items(), key=lambda x: x[1]))
    return {**d}


def get_key(val, my_dict):
    list_of_keys = []
    boolValue = 0
    for key, value in my_dict.items(): 
         if val == value: 
                list_of_keys.append(key)
    if len(list_of_keys) == 0:
        return []
    else:
        return list_of_keys

### Buses

In [17]:
def generateBusOrigins():
    starts = []
    loc = "ABCDEFGHIJKLMNOPQRSTUVWXY"
    for i in range(Fleet_Size):
        starts.append(random.choice(loc))
        loc = loc.replace(starts[-1],'')
    return starts
BusGeneratedOrigins = generateBusOrigins()
BusGeneratedOrigins

['W', 'X', 'H', 'U', 'A', 'V', 'S', 'O', 'N', 'P']

In [18]:
Buses = {np.arange(Fleet_Size).tolist()[i]: {'Dest': {},'Origin': {}, 'Entry': {}, 'Board': {}, 'Alight': {}, 'Routes':{'Person': [], 'Loc': [], 'Time': []}, 'Promise_Board': {}, 'Promise_Alight': {} , 'current': BusGeneratedOrigins[i] } for i in range(Fleet_Size)}    
Buses

{0: {'Dest': {},
  'Origin': {},
  'Entry': {},
  'Board': {},
  'Alight': {},
  'Routes': {'Person': [], 'Loc': [], 'Time': []},
  'Promise_Board': {},
  'Promise_Alight': {},
  'current': 'W'},
 1: {'Dest': {},
  'Origin': {},
  'Entry': {},
  'Board': {},
  'Alight': {},
  'Routes': {'Person': [], 'Loc': [], 'Time': []},
  'Promise_Board': {},
  'Promise_Alight': {},
  'current': 'X'},
 2: {'Dest': {},
  'Origin': {},
  'Entry': {},
  'Board': {},
  'Alight': {},
  'Routes': {'Person': [], 'Loc': [], 'Time': []},
  'Promise_Board': {},
  'Promise_Alight': {},
  'current': 'H'},
 3: {'Dest': {},
  'Origin': {},
  'Entry': {},
  'Board': {},
  'Alight': {},
  'Routes': {'Person': [], 'Loc': [], 'Time': []},
  'Promise_Board': {},
  'Promise_Alight': {},
  'current': 'U'},
 4: {'Dest': {},
  'Origin': {},
  'Entry': {},
  'Board': {},
  'Alight': {},
  'Routes': {'Person': [], 'Loc': [], 'Time': []},
  'Promise_Board': {},
  'Promise_Alight': {},
  'current': 'A'},
 5: {'Dest': {},
  '

In [55]:
PATH = ['A','B','C','D','E']


def getFullPath(arr):
    fullPath = []
    for i, a in enumerate(arr[:-1]):
        x = SP[(SP['Origin'] == a) & (SP['Exit'] == arr[i+1])]['Shortest Path'].item()
        if len(fullPath) == 0:
            fullPath += x
        else:
            fullPath += x[1:]
    return fullPath
print(getFullPath(['A','E','C']))
PATH = getFullPath(['A','E','C'])     
def getPossibleODList(arr):
    ODPairs = []
    IndexPairs = []
    for i, a in enumerate(arr[:-1]):
        for j, b in enumerate(arr[i+1:]):
            ODPairs.append(a + b)
            IndexPairs.append(str(i) + str(j+i+1))
    return ODPairs, IndexPairs
        
ODLIST, IndexPairs = getPossibleODList(PATH)


def getLegs(arr):
    Legs = []
    IndexLegs = []
    for i, a in enumerate(arr[:-1]):
        Legs.append(a + arr[i+1])
        IndexLegs.append(str(i)+str(i+1))
    return Legs, IndexLegs
        

LEGS, IndexLegs = getLegs(PATH)

print(LEGS, IndexLegs, ODLIST, IndexPairs)

['A', 'B', 'C', 'D', 'E', 'D', 'C']
['AB', 'BC', 'CD', 'DE', 'ED', 'DC'] ['01', '12', '23', '34', '45', '56'] ['AB', 'AC', 'AD', 'AE', 'AD', 'AC', 'BC', 'BD', 'BE', 'BD', 'BC', 'CD', 'CE', 'CD', 'CC', 'DE', 'DD', 'DC', 'ED', 'EC', 'DC'] ['01', '02', '03', '04', '05', '06', '12', '13', '14', '15', '16', '23', '24', '25', '26', '34', '35', '36', '45', '46', '56']


In [20]:
numpy_data = np.zeros((len(LEGS),len(ODLIST))).tolist()
MATRIX = pd.DataFrame(data=numpy_data, index=LEGS, columns=ODLIST)
MATRIX

,AB,AC,AD,AE,AD,AC,BC,BD,BE,BD,...,CD,CE,CD,CC,DE,DD,DC,ED,EC,DC
AB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ED,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
pathDict = {}
for i, p in enumerate(PATH):
    try:
        pathDict[p].append(i)
    except:
        pathDict[p] = []
        pathDict[p].append(i)
pathDict

{'A': [0], 'B': [1], 'C': [2, 6], 'D': [3, 5], 'E': [4]}

In [57]:
for i, leg in enumerate(LEGS):
    temp = {}
    for j, od in enumerate(ODLIST):
        try:
            temp[od] += 1
        except:
            temp[od] = 1
        if (int(IndexLegs[i][0]) >= int(IndexPairs[j][0])) and (int(IndexLegs[i][1]) <= int(IndexPairs[j][1])):
            MATRIX.at[leg, od] = 1
MATRIX

,AB,AC,AD,AE,AD,AC,BC,BD,BE,BD,...,CD,CE,CD,CC,DE,DD,DC,ED,EC,DC
AB,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BC,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CD,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
DE,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
ED,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
DC,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0


In [ ]:
passenger, Origin_List, Dest_List, EntryTime, waitingTime, BoardingTime, TravelTime, AlightingTime, Bus_Selected, Buses_Route_Person, Buses_Route_Loc, promiseAlight, promiseBoard = [], [], [], [], [], [], [], [], [], [], [], [], []
Buses = {np.arange(Fleet_Size).tolist()[i]: {'Dest': {},'Origin': {}, 'Entry': {}, 'Board': {}, 'Alight': {}, 'Routes':{'Person': [], 'Loc': [], 'Time': []}, 'Promise_Board': {}, 'Promise_Alight': {} , 'current': BusGeneratedOrigins[i] } for i in range(Fleet_Size)}    
rej, reason, costFunc = [], [], []
# for i in tqdm(range(len(TD))):
for i in range(len(TD)):
    Individual = TD['Passengers'][i]
    Origin = TD['Passenger Origin'][i]
    Dest = TD['Passenger Destination'][i]
    Entry = TD['Entry Time'][i].to_pydatetime()
    
    LastEntry = Entry
    LastDest = Dest
    LastOrigin = Origin
    
    print('################################### ', i)
    print(i, LastEntry, LastOrigin, LastDest)
    display(Buses)
    
    #******************************************************************************************************************
    # Checking if any of the times in the Alight are more than the time recently realized then we will have to 
    # remove all those passengers whose alight time was in between the last demand realized time and this time realized
    # and if the alight time is more than the current realized time the current location of the bus will be the last place 
    # a person was alighted
    
    
    # For all Buses
    #-----------------
    for b in range(Fleet_Size):
        if len(Buses[b]['Dest']) == 0:
            continue
        else:
            assert len(Buses[b]['Dest'].keys()) == len(Buses[b]['Entry'].keys()) == len(Buses[b]['Alight'].keys()) == len(Buses[b]['Board'].keys())
            
            # Find the Current Bus based on the new passenger Entry time.
            # ------------------------------------------------------------
            last_index = 0
            for ii, t in enumerate(Buses[b]['Routes']['Time']):
                if Entry >= t:
                    last_index = ii
                else:
                    break
            Buses[b]['current'] = Buses[b]['Routes']['Loc'][last_index]
            
            # Adding to arrays the passengers whose rides are over
            # ---------------------------------------------------
            busBoard = copy.deepcopy(Buses[b]['Board'])
            for person , time in busBoard.items():
                if Entry >= time and Entry >= Buses[b]['Alight'][person]:
                    passenger.append(person)
                    Bus_Selected.append(b)
                    Origin_List.append(Buses[b]['Origin'][person])
                    Dest_List.append(Buses[b]['Dest'][person])
                    EntryTime.append(Buses[b]['Entry'][person])
                    BoardingTime.append(Buses[b]['Board'][person])
                    AlightingTime.append(Buses[b]['Alight'][person])
                    waitingTime.append(timedelta2time(BoardingTime[-1] - EntryTime[-1]))
                    TravelTime.append(timedelta2time(AlightingTime[-1] - BoardingTime[-1]))
                    promiseAlight.append(Buses[b]['Promise_Alight'][person])
                    promiseBoard.append(Buses[b]['Promise_Board'][person])
                    
                    Buses_Route_Person.append(Buses[b]['Routes']['Person'])
                    Buses_Route_Loc.append(Buses[b]['Routes']['Loc'])
                    
                    try:
                        Buses[b]['Entry'].pop(person)
                        Buses[b]['Board'].pop(person)
                        Buses[b]['Alight'].pop(person)
                        Buses[b]['Dest'].pop(person)
                        Buses[b]['Origin'].pop(person)
                        Buses[b]['Promise_Alight'].pop(person)
                        Buses[b]['Promise_Board'].pop(person)
                    except KeyError:
                        print('Key Not In The Dictionary')    
                elif Entry >= time:
                    continue
                else:
                    break        
            Buses[b]['Alight'] = sortValues(Buses[b]['Alight'])
    #******************************************************************************************************************
    # Here we look at the bus selected for the current demand realized, we first check the cost of without including the person
    # and then we check when we add the person what is the difference between the two costs. The minimum of the two costs bus is
    # selected to pick up that passenger

    
    
    # For all Buses
    #-----------------
    cost_old, cost_new, cost_diff, PathDistances, CapacityFlag = [], [], [], [], []
    for b in range(Fleet_Size):
        if len(Buses[b]['Dest']) >= Bus_Capacity: # If a bus has reached full capacity
            ##########################################################################################################
#             cost_old.append(datetime.timedelta(seconds=0))
#             cost_new.append(datetime.timedelta(seconds=0))
#             cost_diff.append(datetime.timedelta(seconds=99999999))
            cost_old.append(0)
            cost_new.append(0)
            cost_diff.append(99999999)
            path_distances = {'Person': [], 'Loc': [], 'Dist': []}
            PathDistances.append(path_distances)
            CapacityFlag.append(1)
            

        elif len(Buses[b]['Dest']) == 0: # If a bus is empty
            ##########################################################################################################
            Bus_Origin = Buses[b]['current']
            path_distances = {'Person': [], 'Loc': [], 'Dist': []}
            if Origin == Bus_Origin:
                path_distances['Person'] += [Individual]*2
                path_distances['Loc'] += [Origin, Dest]
                path_distances['Dist'] += [0,dist_matrix[Loc2Index[Origin]][Loc2Index[Dest]]]
            else:
                path_distances['Person'].append(-1)
                path_distances['Person'] += [Individual]*2
                path_distances['Loc'] += [Bus_Origin, Origin, Dest]
                BO2O = dist_matrix[Loc2Index[Bus_Origin]][Loc2Index[Origin]]
                BO2D = BO2O + dist_matrix[Loc2Index[Origin]][Loc2Index[Dest]]
                path_distances['Dist'] += [0, BO2O, BO2D]
            
            assert len(path_distances['Dist']) == len(path_distances['Loc']) == len(path_distances['Person'])
            print('PD: ', b, path_distances)
            PathDistances.append(path_distances)

#             cost_old.append(datetime.timedelta(seconds=0))
#             cost_new.append(time2timedelta(toTime(timeCalculation(Uniform_Speed, BO2D))))
#             cost_diff.append(cost_new[-1] - cost_old[-1])
            cost_old.append(0)
            path_distances['Loc']
        
        
            print(cost_diff[-1])
        else: # If a bus is NOT empty
            ##########################################################################################################
            # Old cost before adding a passenger
            # ---------------------------------
            assert len(Buses[b]['Dest'].keys()) == len(Buses[b]['Entry'].keys()) == len(Buses[b]['Alight'].keys())
            passengers = list(Buses[b]['Dest'].keys())
            sum = datetime.timedelta(seconds=0)
            for p in passengers:
                entryTime = datetime2time(Buses[b]['Entry'][p])
                alightTime = datetime2time(Buses[b]['Alight'][p])
                sum = sum + (time2timedelta(alightTime) - time2timedelta(entryTime))
            cost_old.append(sum)
            # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
            
            # Getting 4 columns of the passenger, the origin and the destination and flags including the latest passenger
            Bus_Origin = Buses[b]['current']
            path_distances = {'Person': [], 'Loc': [], 'Dist': []}
            fromLoc, toLoc, person, flags = [], [], [], []
            # Get all origin for ppl on that bus and add their destinations also adding whether they were at Bus Origin then 
            # they should be picked up.
            # --------------------------------------------------------------------------------------
            set_origins = set(list( Buses[b]['Origin'].values()))
            for v in set_origins:
                ppl = get_key(v, Buses[b]['Origin'])
                if len(ppl):
                    for p in ppl:
                        person.append(p)
                        fromLoc.append(v)
                        toLoc.append(Buses[b]['Dest'][p])
                        if Entry > Buses[b]['Board'][p]:
                            flags.append(1)
                        else:
                            flags.append(0)
            person.append(TD['Passengers'][i])
            fromLoc.append(Origin)
            toLoc.append(Dest)
            if Bus_Origin == Origin:
                flags.append(1)
            else:
                flags.append(0)
            assert len(toLoc) == len(fromLoc) == len(person) == len(flags)
            print('List_1: ', person, fromLoc, toLoc, flags)
                             
            indices_not_boarded = [index for index, value in enumerate(flags) if value == 0]
            indices_boarded = [index for index, value in enumerate(flags) if value == 1]
            # Adding the already existing people (onBoard) on the bus for the shortest path algorithm
            # --------------------------------------------------------------------------------------
            Buses[b]['Board'] = sortValues(Buses[b]['Board'])
            for pp, tt in Buses[b]['Board'].items():
                if person.index(pp) in indices_boarded:
                    path_distances['Person'].append(person[person.index(pp)])
                    path_distances['Loc'].append(fromLoc[person.index(pp)])
                    path_distances['Dist'].append(0.0)
            if Bus_Origin == Origin: # If new guy has boarded
                path_distances['Person'].append(TD['Passengers'][i])
                path_distances['Loc'].append(Origin)
                path_distances['Dist'].append(0.0)
            
            assert len(path_distances['Person']) == len(path_distances['Loc']) == len(path_distances['Dist'])
            
            # If bus going to get someone and new guy also wants to get on
            # --------------------------------------------------------------------------------------
            if len(path_distances['Person']) == 0:
                path_distances['Person'].append(-1)
                path_distances['Loc'].append(Bus_Origin)
                path_distances['Dist'].append(0.0)
            
            # Fill the checkpoints with not boarded origins and boarded destinations so we can see if a customer is present here or not
            # --------------------------------------------------------------------------------------
            indices_not_boarded = [index for index, value in enumerate(flags) if value == 0]
            indices_boarded = [index for index, value in enumerate(flags) if value == 1]
            check_points = []
            for ii in indices_not_boarded:
                if fromLoc[ii] not in check_points:
                    check_points.append(fromLoc[ii])
            for ii in indices_boarded:
                if toLoc[ii] not in check_points:
                    check_points.append(toLoc[ii])
            
            # Nearest Neighbour Algorithm 
            # --------------------------------------------------------------------------------------
            place_reached, person_seated = [], []
            Temp_Origin = Bus_Origin
            while len(check_points)!= 0:
                smallest_checkpoint_dist = []
                for cp in check_points:
                    smallest_checkpoint_dist.append(dist_matrix[Loc2Index[Temp_Origin]][Loc2Index[cp]])
                smallest_index = smallest_checkpoint_dist.index(min(smallest_checkpoint_dist)) 
                Temp_Origin = check_points[smallest_index]
                # Add destinations if they have picked up from the origin
                indices_origin = [index for index, value in enumerate(fromLoc) if value == Temp_Origin]
                add_dest_indices = [index for index, value in enumerate(toLoc) if index in indices_origin]
                
                for ii in add_dest_indices:
                    if flags[ii] != 1 and flags[ii] != 2:
                        place_reached.append(Temp_Origin)
                        person_seated.append(person[ii])
                        flags[ii] = 1
                        if toLoc[ii] not in check_points:
                            check_points.append(toLoc[ii])
                
                indices_not_boarded = [index for index, value in enumerate(flags) if value == 0]
                indices_boarded = [index for index, value in enumerate(flags) if value == 1]
                
                dropped_off_indices = [index for index, value in enumerate(toLoc) if value == Temp_Origin and index in indices_boarded]
                for ii in dropped_off_indices:
                    place_reached.append(Temp_Origin)
                    person_seated.append(person[ii])
                    flags[ii] = 2 
                check_points.pop(smallest_index)
            
            origin_boarded =  len(path_distances['Loc'])
            for pp in person_seated:
                path_distances['Person'].append(pp)
            for ll in place_reached:
                path_distances['Loc'].append(ll)
            mini_dist = 0
            Temp_Origin = path_distances['Loc'][0]
            if path_distances['Person'][0] == -1:
                for Location in  path_distances['Loc'][1:]:
                    mini_dist = mini_dist + dist_matrix[Loc2Index[Temp_Origin]][Loc2Index[Location]]
                    path_distances['Dist'] += [mini_dist]
                    Temp_Origin = Location
            else:
                for Location in  path_distances['Loc'][origin_boarded:]:
                    mini_dist = mini_dist + dist_matrix[Loc2Index[Temp_Origin]][Loc2Index[Location]]
                    path_distances['Dist'] += [mini_dist]
                    Temp_Origin = Location
            
            assert len(path_distances['Dist']) == len(path_distances['Loc']) == len(path_distances['Person'])
            print('PD: ',b, path_distances, '-------')
            PathDistances.append(path_distances)
            #-------------------------------------------------------------------------------
            # Find the cost of adding the new passenger when adding him to the list of current passengers
            wontPick = 0
            sum = datetime.timedelta(seconds=0)
            for p in passengers:
                entryTime = Buses[b]['Entry'][p]
                enter = Buses[b]['Origin'][p]
                exit = Buses[b]['Dest'][p]
                # get the index for the passenger as to where he got on and where he got off and the boarding and alight time
                # ---------------------------------------------------------------------------
                enter_index, exit_index = 0, 0
                count_occurence = 0
                for ii, pp in enumerate(path_distances['Person']):
                    if pp == p and count_occurence == 1:
                        count_occurence += 1
                        exit_index = ii
                    elif pp == p:
                        count_occurence += 1
                        enter_index = ii
                    else:
                        continue
                assert count_occurence == 2
                
                assert path_distances['Loc'][enter_index] == enter
                assert path_distances['Loc'][exit_index] == exit
                boardTime = Entry + time2timedelta(toTime(timeCalculation(Uniform_Speed, path_distances['Dist'][enter_index])))
                alightTime = Entry + time2timedelta(toTime(timeCalculation(Uniform_Speed, path_distances['Dist'][exit_index])))
                # Make sure the travel time and waiting time of each person doesnt cross the limit
                if boardTime > Buses[b]['Promise_Board'][p]:
                    wontPick = 1
                if alightTime > Buses[b]['Promise_Alight'][p]:
                    wontPick = 2    
                sum = sum + (time2timedelta(datetime2time(alightTime)) - time2timedelta(datetime2time(Buses[b]['Entry'][p])))
            
            count_occurence = 0
            for ii, pp in enumerate(path_distances['Person']):
                if pp == TD['Passengers'][i] and count_occurence == 1:
                    count_occurence += 1
                    exit_index = ii
                elif pp == TD['Passengers'][i]:
                    count_occurence += 1
                    enter_index = ii
                else:
                    continue
            assert count_occurence == 2
            alightTime = Entry + time2timedelta(toTime(timeCalculation(Uniform_Speed, path_distances['Dist'][exit_index])))
            sum = sum + (time2timedelta(datetime2time(alightTime)) - time2timedelta(datetime2time(Entry)))
            cost_new.append(sum)
            if (wontPick == 1) or (wontPick == 2):
                if wontPick == 1:
                    cost_diff.append(datetime.timedelta(seconds=99999998))
                elif wontPick == 2:
                    cost_diff.append(datetime.timedelta(seconds=99999997))
            else:
                cost_diff.append(cost_new[-1] - cost_old[-1])
            print(cost_diff[-1])
                                           
    #******************************************************************************************************************
    # If the travelling time for any customer is exceeded because of addition of new passenger and this be the case for all Buses

    assert len(cost_diff) == len(cost_old) == len(cost_new)
    if min(cost_diff) >= datetime.timedelta(seconds=99999997):
        statement = set()
        for cost in cost_diff:
            if cost == datetime.timedelta(seconds=99999997):
                statement.add('Alight Time')
            elif cost == datetime.timedelta(seconds=99999998):
                statement.add('Board Time')
            elif cost == datetime.timedelta(seconds=99999999):
                statement.add('Bus Capacity')
#         print('Customer {} has been rejected'.format(i))
        reason.append(statement)
        rej.append(TD['Passengers'][i])
        continue
    bus_index = cost_diff.index(min(cost_diff))
    costFunc.append(cost_diff[bus_index])
    path_distances = PathDistances[bus_index]

    # Here we will update the bus that has been selected to pick up the passenger, that means addition of the customer
    # and adding his entry time, calculating his boarding time and all the alighting time after the route change
    print('PATH DISTANCE SELECTED: ',bus_index ,path_distances)
    if [1]* Fleet_Size != CapacityFlag: # If all Buses haven't reached full capacity
    
        Buses[bus_index]['Dest'][TD['Passengers'][i]] = Dest
        Buses[bus_index]['Entry'][TD['Passengers'][i]] = Entry
        Buses[bus_index]['Origin'][TD['Passengers'][i]] = Origin
        Bus_Origin = Buses[bus_index]['current']
        
        for ppl in list(Buses[bus_index]['Origin'].keys()):
            loc = Buses[bus_index]['Origin'][ppl]
            enter_index, exit_index, count_occurence = 0, 0, 0
            for ii, pp in enumerate(path_distances['Person']):
                if pp == ppl and count_occurence == 1:
                    count_occurence += 1
                    exit_index = ii
                elif pp == ppl:
                    count_occurence += 1
                    enter_index = ii
                else:
                    continue
            assert count_occurence == 2
            Buses[bus_index]['Board'][ppl] = Entry + time2timedelta(toTime(timeCalculation(Uniform_Speed, path_distances['Dist'][enter_index])))
            if ppl == TD['Passengers'][i]:
                Buses[bus_index]['Promise_Board'][TD['Passengers'][i]] = Buses[bus_index]['Board'][TD['Passengers'][i]] + Waiting_Time_Threshold
        
        for ppl in list(Buses[bus_index]['Dest'].keys()):
            loc = Buses[bus_index]['Dest'][ppl]
            enter_index, exit_index, count_occurence = 0, 0, 0
            for ii, pp in enumerate(path_distances['Person']):
                if pp == ppl and count_occurence == 1:
                    count_occurence += 1
                    exit_index = ii
                elif pp == ppl:
                    count_occurence += 1
                    enter_index = ii
                else:
                    continue
            assert count_occurence == 2
            Buses[bus_index]['Alight'][ppl] = Entry + time2timedelta(toTime(timeCalculation(Uniform_Speed, path_distances['Dist'][exit_index])))
            if ppl == TD['Passengers'][i]:
                Buses[bus_index]['Promise_Alight'][TD['Passengers'][i]] = Buses[bus_index]['Alight'][TD['Passengers'][i]] + Travel_Time_Threshold
        
        
        assert len(path_distances['Dist']) == len(path_distances['Loc']) == len(path_distances['Person'])
        
        # Forming the routes and their distances
        startingDistIndex = 0
        for ii, dd in enumerate(path_distances['Dist']):
            if dd != 0.0:
                startingDistIndex = ii - 1
                break
        
        thePerson = path_distances['Person'][startingDistIndex]
        theLoc = path_distances['Loc'][startingDistIndex]
        
        Temp = path_distances['Loc'][0]
        Person_List = [path_distances['Person'][0]]
        person_index = 1
        Mini_Path = [Temp]
        for ll in path_distances['Loc'][1:]:
            numInBetween = len(ConstructPath(predecessors, Loc2Index[Temp], Loc2Index[ll])[1:]) - 1
            Person_List += ([-1] * numInBetween)
            Person_List += [path_distances['Person'][person_index]]
            if len(ConstructPath(predecessors, Loc2Index[Temp], Loc2Index[ll])) == 1:
                Mini_Path += ConstructPath(predecessors, Loc2Index[Temp], Loc2Index[ll])
            else:
                Mini_Path += ConstructPath(predecessors, Loc2Index[Temp], Loc2Index[ll])[1:]
            Temp = ll
            person_index += 1
        
        try:
            assert len(Mini_Path) == len(Person_List)
        except:
            print(Mini_Path, Person_List)
        assert len(Mini_Path) == len(Person_List)
        
        save_index = -1
        for ii, ee in enumerate(Mini_Path):
            if (ee == theLoc) and (Person_List[ii] == thePerson):
                save_index = ii
                break
        
        Mini_Dist = 0
        Temp = Mini_Path[0]
        Dist_List = [0.0]
        for ii, ll in enumerate(Mini_Path[1:]):
            if ii + 1 <= save_index:
                Dist_List.append(0.0)
            else:
                Dist_List.append(Mini_Dist + dist_matrix[Loc2Index[Temp]][Loc2Index[ll]])
            Mini_Dist = Mini_Dist + dist_matrix[Loc2Index[Temp]][Loc2Index[ll]]
            Temp = ll

        assert len(Mini_Path) == len(Person_List) == len(Dist_List)
        
        print('Person', Person_List)
        print('Location', Mini_Path)
        print('Distance', Dist_List)
    
        Buses[bus_index]['Routes']['Person'] = Person_List
        Buses[bus_index]['Routes']['Loc'] = Mini_Path
        Buses[bus_index]['Routes']['Time'] = [Entry + time2timedelta(toTime(timeCalculation(Uniform_Speed, d))) for d in Dist_List]
        Buses[bus_index]['Alight'] = sortValues(Buses[bus_index]['Alight'])
    else:
        print('All Buses have reached full capacity.')